# Futures price prediction with classifiers 

The idea of the analysis is to check whether classification algorithms could be useful in FW20 futures price predictions.
Algorithms used in the analysis:
- Logistic Regression
- Support Vector Classifier
- K-Nearest Neighbors
- Random Forest Classifier
- Ada Boost Classifier (with Decision Trees)
- Voting Classifier

Dependent variable:
- Price change in the current day (D) - from open to close (0 for downtrend, 1 for uptrend)

Predictors:
- Rate of return (%) in the previous day (D-1),
- Change in volume (%) in the previous day (D-1),
- Change in open interest (%) in the previous day (D-1).

Validation dataset consists of previous 2000 sessions. For each algorithm 2000 models have been trained, based on data from previous 1000 sessions. Each model has only one session in test set (that comes after training dataset). Efficiency of algorithms is evaluated by accuracy measure (true positives + true negatives / sum of predictions). However, there are also results of profit/loss (in points) printed for each model. In this approch we can consider each algorithm as a simple trading system, in which buy/short sell signals are generated at the opening of each session.

In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
data_fw20 = pd.read_excel('stocks_data/FW20.xlsx', index_col='Data')

In [4]:
data_fw20.rename(columns={"FW20_Otwarcie":"Open", "FW20_Zamkniecie":"Close", 
                          "FW20_Wolumen":"Volume", "FW20_LOP":"OpenInt"}, inplace=True)

In [5]:
data_fw20.head()

,Open,Close,Volume,OpenInt
Data,,,,
2000-01-03,1937,1920,4070,4955.0
2000-01-04,1865,1847,4255,4936.0
2000-01-05,1827,1811,5172,4694.0
2000-01-06,1802,1848,5220,4801.0
2000-01-07,1936,1982,5671,4918.0


## Data preprocessing

Create function that computes changes in values (%) for specified columns for the previous day (change from D-2 to D-1).

In [6]:
def change_previous_day(*args):
    for arg in args:
        data_fw20[arg +'_ret_-1'] = (data_fw20[arg].shift(periods=1) / 
                                      data_fw20[arg].shift(periods=2)-1)*100

Apply change_previous_day() function to columns: Close, Volume and OpenInterest

In [7]:
change_previous_day('Close', 'Volume', 'OpenInt')

In [8]:
data_fw20.tail()

,Open,Close,Volume,OpenInt,Close_ret_-1,Volume_ret_-1,OpenInt_ret_-1
Data,,,,,,,
2018-12-19,2317,2350,39377,60337.0,1.577564,57.106927,3.689072
2018-12-20,2330,2320,41267,54762.0,1.380500,-9.946028,6.429479
2018-12-21,2313,2275,32056,56332.0,-1.276596,4.799756,-9.239770
2018-12-27,2295,2255,16556,48047.0,-1.939655,-22.320498,2.866952
2018-12-28,2262,2278,11235,47229.0,-0.879121,-48.352882,-14.707449


Create column with direction (change from Open to Close in current day): 1 for uptrend, 0 for downtrend

In [9]:
data_fw20['FW20_dir_o_c'] = np.where(data_fw20['Close'] > data_fw20['Open'], 1, 0)

Create final set of data for analysis

In [10]:
data_fw20_all = data_fw20[['Open','Close','Close_ret_-1','Volume_ret_-1','OpenInt_ret_-1','FW20_dir_o_c']]

Drop few columns with missing data

In [11]:
data_fw20_all.dropna(inplace=True)

Shape of dataframe and names of columns

In [12]:
data_fw20_all.shape

(4751, 6)

In [13]:
data_fw20_all.columns

Index(['Open', 'Close', 'Close_ret_-1', 'Volume_ret_-1', 'OpenInt_ret_-1',
       'FW20_dir_o_c'],
      dtype='object')

In [14]:
data_fw20_all.tail()

,Open,Close,Close_ret_-1,Volume_ret_-1,OpenInt_ret_-1,FW20_dir_o_c
Data,,,,,,
2018-12-19,2317,2350,1.577564,57.106927,3.689072,1
2018-12-20,2330,2320,1.380500,-9.946028,6.429479,0
2018-12-21,2313,2275,-1.276596,4.799756,-9.239770,0
2018-12-27,2295,2255,-1.939655,-22.320498,2.866952,0
2018-12-28,2262,2278,-0.879121,-48.352882,-14.707449,1


## Analysis

For each model there is loop testing xxx datasets - results are collected in a lists and finally a summary with accuracy and results (in pts.) is printed.

### 1. Logistic regression

Increase of regularization (by reducing 'C' parameter to 0,001) resulted in better accuracy compared to default parameters.

In [15]:
train_period = 1000
test_period = 2000
date, predictions, y_actual, pred_proba, results = [], [], [], [], []
model = LogisticRegression(C=0.001)

for t in range(test_period):
    X_train = data_fw20_all.iloc[(-train_period - t - 1) : (-t - 1)][['Close_ret_-1', 'Volume_ret_-1', 
                                                                      'OpenInt_ret_-1']]
    y_train = data_fw20_all.iloc[(-train_period - t - 1) : (-t - 1)][['FW20_dir_o_c']]
    X_test = data_fw20_all.iloc[-t - 1][['Close_ret_-1', 'Volume_ret_-1', 'OpenInt_ret_-1']]
    y_test = data_fw20_all.iloc[-t - 1]['FW20_dir_o_c']
    
    model.fit(X_train, y_train)
    prediction = model.predict(np.array(X_test).reshape(1,-1))
    prediction_proba = model.predict_proba(np.array(X_test).reshape(1,-1))
    date.append(np.datetime64(data_fw20_all.index[-t -1], 'D'))
    y_actual.append(int(y_test))
    predictions.append(prediction[0])
    pred_proba.append(prediction_proba)
    
    result = abs(data_fw20_all.iloc[-t - 1]['Close'] - data_fw20_all.iloc[-t - 1]['Open'])
    results.append(result if y_test == prediction else -result)

accuracy = 0
for i in range(len(predictions)):
    if y_actual[i] == predictions[i]:
        accuracy+=1
print('Model summary')
print('Accuracy: {}'.format(round(accuracy / len(predictions),4)))
print('Result (pts.): {}'.format(sum(results)))
print('Average result per 1 session: {}'.format(sum(results) / test_period))
print('*'*100)

for i in range(len(predictions)):
    print(date[i], '\t', 'actual:', y_actual[i], predictions[i], ':prediction', 'preds_proba:', pred_proba[i],
         '\t', 'result:', results[i])

Model summary
Accuracy: 0.534
Result (pts.): 2366.0
Average result per 1 session: 1.183
****************************************************************************************************
2018-12-28 	 actual: 1 0 :prediction preds_proba: [[0.50572779 0.49427221]] 	 result: -16.0
2018-12-27 	 actual: 0 1 :prediction preds_proba: [[0.49165713 0.50834287]] 	 result: -40.0
2018-12-21 	 actual: 0 0 :prediction preds_proba: [[0.51298638 0.48701362]] 	 result: 38.0
2018-12-20 	 actual: 0 0 :prediction preds_proba: [[0.50310188 0.49689812]] 	 result: 10.0
2018-12-19 	 actual: 1 0 :prediction preds_proba: [[0.52117547 0.47882453]] 	 result: -33.0
2018-12-18 	 actual: 1 0 :prediction preds_proba: [[0.50666256 0.49333744]] 	 result: -46.0
2018-12-17 	 actual: 0 0 :prediction preds_proba: [[0.50660023 0.49339977]] 	 result: 22.0
2018-12-14 	 actual: 1 0 :prediction preds_proba: [[0.50892725 0.49107275]] 	 result: -13.0
2018-12-13 	 actual: 1 0 :prediction preds_proba: [[0.51748944 0.48251056]] 	 

2018-06-27 	 actual: 0 0 :prediction preds_proba: [[0.50554237 0.49445763]] 	 result: 10.0
2018-06-26 	 actual: 0 0 :prediction preds_proba: [[0.50661545 0.49338455]] 	 result: 6.0
2018-06-25 	 actual: 1 0 :prediction preds_proba: [[0.50340425 0.49659575]] 	 result: -2.0
2018-06-22 	 actual: 1 0 :prediction preds_proba: [[0.50761719 0.49238281]] 	 result: -3.0
2018-06-21 	 actual: 0 0 :prediction preds_proba: [[0.51863955 0.48136045]] 	 result: 8.0
2018-06-20 	 actual: 1 0 :prediction preds_proba: [[0.50535077 0.49464923]] 	 result: -29.0
2018-06-19 	 actual: 1 0 :prediction preds_proba: [[0.52679837 0.47320163]] 	 result: -6.0
2018-06-18 	 actual: 0 1 :prediction preds_proba: [[0.48885245 0.51114755]] 	 result: -60.0
2018-06-15 	 actual: 0 0 :prediction preds_proba: [[0.50615659 0.49384341]] 	 result: 33.0
2018-06-14 	 actual: 1 0 :prediction preds_proba: [[0.52269057 0.47730943]] 	 result: -9.0
2018-06-13 	 actual: 0 0 :prediction preds_proba: [[0.50096742 0.49903258]] 	 result: 13.0

2017-12-12 	 actual: 1 0 :prediction preds_proba: [[0.50502972 0.49497028]] 	 result: -21.0
2017-12-11 	 actual: 0 0 :prediction preds_proba: [[0.50796534 0.49203466]] 	 result: 30.0
2017-12-08 	 actual: 0 0 :prediction preds_proba: [[0.50768394 0.49231606]] 	 result: 9.0
2017-12-07 	 actual: 0 0 :prediction preds_proba: [[0.50278557 0.49721443]] 	 result: 1.0
2017-12-06 	 actual: 0 0 :prediction preds_proba: [[0.50694404 0.49305596]] 	 result: 0.0
2017-12-05 	 actual: 0 0 :prediction preds_proba: [[0.50628466 0.49371534]] 	 result: 1.0
2017-12-04 	 actual: 0 0 :prediction preds_proba: [[0.50880381 0.49119619]] 	 result: 0.0
2017-12-01 	 actual: 0 0 :prediction preds_proba: [[0.50210142 0.49789858]] 	 result: 26.0
2017-11-30 	 actual: 0 0 :prediction preds_proba: [[0.51099959 0.48900041]] 	 result: 15.0
2017-11-29 	 actual: 0 0 :prediction preds_proba: [[0.50358883 0.49641117]] 	 result: 63.0
2017-11-28 	 actual: 0 0 :prediction preds_proba: [[0.50742145 0.49257855]] 	 result: 6.0
2017

2017-05-24 	 actual: 1 0 :prediction preds_proba: [[0.50573476 0.49426524]] 	 result: -25.0
2017-05-23 	 actual: 0 0 :prediction preds_proba: [[0.50094707 0.49905293]] 	 result: 30.0
2017-05-22 	 actual: 0 0 :prediction preds_proba: [[0.5187753 0.4812247]] 	 result: 1.0
2017-05-19 	 actual: 1 1 :prediction preds_proba: [[0.4735775 0.5264225]] 	 result: 18.0
2017-05-18 	 actual: 0 0 :prediction preds_proba: [[0.50744774 0.49255226]] 	 result: 6.0
2017-05-17 	 actual: 0 1 :prediction preds_proba: [[0.47523943 0.52476057]] 	 result: -14.0
2017-05-16 	 actual: 0 0 :prediction preds_proba: [[0.50534021 0.49465979]] 	 result: 48.0
2017-05-15 	 actual: 0 0 :prediction preds_proba: [[0.52034028 0.47965972]] 	 result: 14.0
2017-05-12 	 actual: 0 0 :prediction preds_proba: [[0.50366081 0.49633919]] 	 result: 9.0
2017-05-11 	 actual: 0 0 :prediction preds_proba: [[0.5052634 0.4947366]] 	 result: 22.0
2017-05-10 	 actual: 0 1 :prediction preds_proba: [[0.49524249 0.50475751]] 	 result: -28.0
2017-

2016-11-04 	 actual: 0 0 :prediction preds_proba: [[0.52366034 0.47633966]] 	 result: 1.0
2016-11-03 	 actual: 1 1 :prediction preds_proba: [[0.48108404 0.51891596]] 	 result: 3.0
2016-11-02 	 actual: 0 0 :prediction preds_proba: [[0.5102157 0.4897843]] 	 result: 39.0
2016-10-31 	 actual: 0 0 :prediction preds_proba: [[0.50416194 0.49583806]] 	 result: 5.0
2016-10-28 	 actual: 1 1 :prediction preds_proba: [[0.49896471 0.50103529]] 	 result: 28.0
2016-10-27 	 actual: 1 0 :prediction preds_proba: [[0.51884754 0.48115246]] 	 result: -12.0
2016-10-26 	 actual: 0 1 :prediction preds_proba: [[0.49762988 0.50237012]] 	 result: -2.0
2016-10-25 	 actual: 1 1 :prediction preds_proba: [[0.47637 0.52363]] 	 result: 2.0
2016-10-24 	 actual: 1 0 :prediction preds_proba: [[0.51081956 0.48918044]] 	 result: -27.0
2016-10-21 	 actual: 1 0 :prediction preds_proba: [[0.5239544 0.4760456]] 	 result: -6.0
2016-10-20 	 actual: 0 0 :prediction preds_proba: [[0.51443767 0.48556233]] 	 result: 2.0
2016-10-19 	

...

2013-07-26 	 actual: 0 0 :prediction preds_proba: [[0.50353894 0.49646106]] 	 result: 17.0
2013-07-25 	 actual: 0 0 :prediction preds_proba: [[0.51803545 0.48196455]] 	 result: 7.0
2013-07-24 	 actual: 1 0 :prediction preds_proba: [[0.50298992 0.49701008]] 	 result: -7.0
2013-07-23 	 actual: 0 1 :prediction preds_proba: [[0.49722527 0.50277473]] 	 result: -16.0
2013-07-22 	 actual: 1 1 :prediction preds_proba: [[0.4979425 0.5020575]] 	 result: 1.0
2013-07-19 	 actual: 1 1 :prediction preds_proba: [[0.49992569 0.50007431]] 	 result: 18.0
2013-07-18 	 actual: 0 0 :prediction preds_proba: [[0.52819275 0.47180725]] 	 result: 12.0
2013-07-17 	 actual: 1 0 :prediction preds_proba: [[0.52314788 0.47685212]] 	 result: -16.0
2013-07-16 	 actual: 0 1 :prediction preds_proba: [[0.49873516 0.50126484]] 	 result: -10.0
2013-07-15 	 actual: 1 1 :prediction preds_proba: [[0.49997573 0.50002427]] 	 result: 6.0
2013-07-12 	 actual: 1 1 :prediction preds_proba: [[0.4828831 0.5171169]] 	 result: 10.0
201

2013-01-09 	 actual: 0 0 :prediction preds_proba: [[0.50173422 0.49826578]] 	 result: 2.0
2013-01-08 	 actual: 0 0 :prediction preds_proba: [[0.51053431 0.48946569]] 	 result: 8.0
2013-01-07 	 actual: 0 0 :prediction preds_proba: [[0.51667816 0.48332184]] 	 result: 22.0
2013-01-04 	 actual: 0 1 :prediction preds_proba: [[0.49651472 0.50348528]] 	 result: -24.0
2013-01-03 	 actual: 1 0 :prediction preds_proba: [[0.54045986 0.45954014]] 	 result: -5.0
2013-01-02 	 actual: 0 0 :prediction preds_proba: [[0.51717193 0.48282807]] 	 result: 8.0
2012-12-28 	 actual: 0 1 :prediction preds_proba: [[0.23518487 0.76481513]] 	 result: -20.0
2012-12-27 	 actual: 1 1 :prediction preds_proba: [[0.49889922 0.50110078]] 	 result: 19.0
2012-12-21 	 actual: 0 1 :prediction preds_proba: [[0.49401339 0.50598661]] 	 result: -3.0
2012-12-20 	 actual: 1 1 :prediction preds_proba: [[0.45777666 0.54222334]] 	 result: 5.0
2012-12-19 	 actual: 1 1 :prediction preds_proba: [[0.45734933 0.54265067]] 	 result: 21.0
2

2012-06-20 	 actual: 0 0 :prediction preds_proba: [[0.52137064 0.47862936]] 	 result: 15.0
2012-06-19 	 actual: 1 1 :prediction preds_proba: [[0.33866304 0.66133696]] 	 result: 56.0
2012-06-18 	 actual: 0 0 :prediction preds_proba: [[0.51246627 0.48753373]] 	 result: 24.0
2012-06-15 	 actual: 1 0 :prediction preds_proba: [[0.52963502 0.47036498]] 	 result: -23.0
2012-06-14 	 actual: 0 0 :prediction preds_proba: [[0.51811683 0.48188317]] 	 result: 11.0
2012-06-13 	 actual: 0 0 :prediction preds_proba: [[0.5247458 0.4752542]] 	 result: 8.0
2012-06-12 	 actual: 1 0 :prediction preds_proba: [[0.50863428 0.49136572]] 	 result: -51.0
2012-06-11 	 actual: 1 0 :prediction preds_proba: [[0.50552487 0.49447513]] 	 result: -4.0
2012-06-08 	 actual: 1 0 :prediction preds_proba: [[0.52097055 0.47902945]] 	 result: -69.0
2012-06-06 	 actual: 1 0 :prediction preds_proba: [[0.50849899 0.49150101]] 	 result: -50.0
2012-06-05 	 actual: 0 1 :prediction preds_proba: [[0.48735716 0.51264284]] 	 result: -34

2011-12-01 	 actual: 0 0 :prediction preds_proba: [[0.54827696 0.45172304]] 	 result: 22.0
2011-11-30 	 actual: 1 0 :prediction preds_proba: [[0.5182728 0.4817272]] 	 result: -115.0
2011-11-29 	 actual: 0 0 :prediction preds_proba: [[0.51604012 0.48395988]] 	 result: 20.0
2011-11-28 	 actual: 1 0 :prediction preds_proba: [[0.51556554 0.48443446]] 	 result: -17.0
2011-11-25 	 actual: 1 0 :prediction preds_proba: [[0.50074919 0.49925081]] 	 result: -29.0
2011-11-24 	 actual: 0 0 :prediction preds_proba: [[0.50653114 0.49346886]] 	 result: 0.0
2011-11-23 	 actual: 0 1 :prediction preds_proba: [[0.4972319 0.5027681]] 	 result: -26.0
2011-11-22 	 actual: 0 0 :prediction preds_proba: [[0.52736952 0.47263048]] 	 result: 3.0
2011-11-21 	 actual: 0 0 :prediction preds_proba: [[0.51656537 0.48343463]] 	 result: 65.0
2011-11-18 	 actual: 0 0 :prediction preds_proba: [[0.51313624 0.48686376]] 	 result: 23.0
2011-11-17 	 actual: 0 1 :prediction preds_proba: [[0.49426773 0.50573227]] 	 result: -30.0

2011-03-09 	 actual: 0 0 :prediction preds_proba: [[0.50972654 0.49027346]] 	 result: 9.0
2011-03-08 	 actual: 0 0 :prediction preds_proba: [[0.5136605 0.4863395]] 	 result: 11.0
2011-03-07 	 actual: 1 0 :prediction preds_proba: [[0.50405148 0.49594852]] 	 result: -20.0
2011-03-04 	 actual: 1 0 :prediction preds_proba: [[0.53243235 0.46756765]] 	 result: -11.0
2011-03-03 	 actual: 1 0 :prediction preds_proba: [[0.51378197 0.48621803]] 	 result: -58.0
2011-03-02 	 actual: 1 0 :prediction preds_proba: [[0.50611075 0.49388925]] 	 result: -34.0
2011-03-01 	 actual: 0 0 :prediction preds_proba: [[0.51959961 0.48040039]] 	 result: 29.0
2011-02-28 	 actual: 1 0 :prediction preds_proba: [[0.50103955 0.49896045]] 	 result: -35.0
2011-02-25 	 actual: 0 0 :prediction preds_proba: [[0.50574044 0.49425956]] 	 result: 1.0
2011-02-24 	 actual: 1 0 :prediction preds_proba: [[0.5084312 0.4915688]] 	 result: -1.0
2011-02-23 	 actual: 1 0 :prediction preds_proba: [[0.5160391 0.4839609]] 	 result: -13.0
2

### 2. Support Vector Classification

Increase of regularization resulted in slightly better accuracy compared to default parameters, but all of predictions were one-way - indicating downtrend (0). Hence, no regularization was applied.
SVM Classifier requires rescaling of the data (with StandardScaler, separately for each training dataset) 

In [16]:
train_period = 1000
test_period = 2000
date, predictions, y_actual, results = [], [], [], []
model = SVC()
scaler = StandardScaler()

for t in range(test_period):
    X_train = data_fw20_all.iloc[(-train_period - t - 1) : (-t - 1)][['Close_ret_-1', 'Volume_ret_-1', 
                                                                      'OpenInt_ret_-1']]
    y_train = data_fw20_all.iloc[(-train_period - t - 1) : (-t - 1)][['FW20_dir_o_c']]
    X_test = data_fw20_all.iloc[-t - 1][['Close_ret_-1', 'Volume_ret_-1', 'OpenInt_ret_-1']]
    y_test = data_fw20_all.iloc[-t - 1]['FW20_dir_o_c']
    
    X_train_sc = scaler.fit_transform(X_train)
    X_test_sc = scaler.transform(np.array(X_test).reshape(1,-1))
    
    model.fit(X_train_sc, y_train)
    prediction = model.predict(np.array(X_test_sc).reshape(1,-1))
    date.append(np.datetime64(data_fw20_all.index[-t -1], 'D'))
    y_actual.append(int(y_test))
    predictions.append(prediction[0])
    
    result = abs(data_fw20_all.iloc[-t - 1]['Close'] - data_fw20_all.iloc[-t - 1]['Open'])
    results.append(result if y_test == prediction else -result)

accuracy = 0
for i in range(len(predictions)):
    if y_actual[i] == predictions[i]:
        accuracy+=1
print('Model summary')
print('Accuracy: {}'.format(round(accuracy / len(predictions),4)))
print('Result (pts.): {}'.format(sum(results)))
print('Average result per 1 session: {}'.format(sum(results) / test_period))
print('*'*100)

for i in range(len(predictions)):
    print(date[i], '\t', 'actual:', y_actual[i], predictions[i], ':prediction', '\t', 'result:', results[i])

Model summary
Accuracy: 0.5315
Result (pts.): 1758.0
Average result per 1 session: 0.879
****************************************************************************************************
2018-12-28 	 actual: 1 1 :prediction 	 result: 16.0
2018-12-27 	 actual: 0 1 :prediction 	 result: -40.0
2018-12-21 	 actual: 0 1 :prediction 	 result: -38.0
2018-12-20 	 actual: 0 0 :prediction 	 result: 10.0
2018-12-19 	 actual: 1 0 :prediction 	 result: -33.0
2018-12-18 	 actual: 1 0 :prediction 	 result: -46.0
2018-12-17 	 actual: 0 0 :prediction 	 result: 22.0
2018-12-14 	 actual: 1 0 :prediction 	 result: -13.0
2018-12-13 	 actual: 1 0 :prediction 	 result: -21.0
2018-12-12 	 actual: 1 1 :prediction 	 result: 53.0
2018-12-11 	 actual: 0 1 :prediction 	 result: -48.0
2018-12-10 	 actual: 0 0 :prediction 	 result: 30.0
2018-12-07 	 actual: 0 1 :prediction 	 result: -25.0
2018-12-06 	 actual: 0 0 :prediction 	 result: 29.0
2018-12-05 	 actual: 1 0 :prediction 	 result: -38.0
2018-12-04 	 actual: 

2018-02-26 	 actual: 0 0 :prediction 	 result: 8.0
2018-02-23 	 actual: 0 0 :prediction 	 result: 12.0
2018-02-22 	 actual: 1 0 :prediction 	 result: -29.0
2018-02-21 	 actual: 0 0 :prediction 	 result: 8.0
2018-02-20 	 actual: 1 0 :prediction 	 result: -1.0
2018-02-19 	 actual: 0 0 :prediction 	 result: 11.0
2018-02-16 	 actual: 0 1 :prediction 	 result: -33.0
2018-02-15 	 actual: 0 0 :prediction 	 result: 47.0
2018-02-14 	 actual: 1 0 :prediction 	 result: -7.0
2018-02-13 	 actual: 1 0 :prediction 	 result: -15.0
2018-02-12 	 actual: 1 0 :prediction 	 result: -3.0
2018-02-09 	 actual: 1 1 :prediction 	 result: 2.0
2018-02-08 	 actual: 0 1 :prediction 	 result: -25.0
2018-02-07 	 actual: 1 0 :prediction 	 result: -17.0
2018-02-06 	 actual: 0 0 :prediction 	 result: 33.0
2018-02-05 	 actual: 1 0 :prediction 	 result: -12.0
2018-02-02 	 actual: 0 0 :prediction 	 result: 13.0
2018-02-01 	 actual: 0 0 :prediction 	 result: 32.0
2018-01-31 	 actual: 0 0 :prediction 	 result: 17.0
2018-01-3

2017-06-29 	 actual: 0 0 :prediction 	 result: 18.0
2017-06-28 	 actual: 0 0 :prediction 	 result: 7.0
2017-06-27 	 actual: 1 0 :prediction 	 result: -12.0
2017-06-26 	 actual: 1 0 :prediction 	 result: -21.0
2017-06-23 	 actual: 1 0 :prediction 	 result: -9.0
2017-06-22 	 actual: 0 0 :prediction 	 result: 13.0
2017-06-21 	 actual: 1 0 :prediction 	 result: -22.0
2017-06-20 	 actual: 0 1 :prediction 	 result: -25.0
2017-06-19 	 actual: 1 0 :prediction 	 result: -12.0
2017-06-16 	 actual: 1 0 :prediction 	 result: -13.0
2017-06-14 	 actual: 0 1 :prediction 	 result: -15.0
2017-06-13 	 actual: 1 0 :prediction 	 result: -2.0
2017-06-12 	 actual: 0 0 :prediction 	 result: 39.0
2017-06-09 	 actual: 0 1 :prediction 	 result: -16.0
2017-06-08 	 actual: 1 0 :prediction 	 result: -35.0
2017-06-07 	 actual: 0 1 :prediction 	 result: -7.0
2017-06-06 	 actual: 1 0 :prediction 	 result: -45.0
2017-06-05 	 actual: 0 0 :prediction 	 result: 31.0
2017-06-02 	 actual: 0 0 :prediction 	 result: 7.0
2017

2016-10-26 	 actual: 0 0 :prediction 	 result: 2.0
2016-10-25 	 actual: 1 0 :prediction 	 result: -2.0
2016-10-24 	 actual: 1 0 :prediction 	 result: -27.0
2016-10-21 	 actual: 1 0 :prediction 	 result: -6.0
2016-10-20 	 actual: 0 0 :prediction 	 result: 2.0
2016-10-19 	 actual: 1 0 :prediction 	 result: -29.0
2016-10-18 	 actual: 1 0 :prediction 	 result: -6.0
2016-10-17 	 actual: 0 0 :prediction 	 result: 2.0
2016-10-14 	 actual: 0 1 :prediction 	 result: -10.0
2016-10-13 	 actual: 0 0 :prediction 	 result: 22.0
2016-10-12 	 actual: 1 0 :prediction 	 result: -9.0
2016-10-11 	 actual: 0 0 :prediction 	 result: 13.0
2016-10-10 	 actual: 1 0 :prediction 	 result: -8.0
2016-10-07 	 actual: 0 0 :prediction 	 result: 5.0
2016-10-06 	 actual: 0 0 :prediction 	 result: 5.0
2016-10-05 	 actual: 1 1 :prediction 	 result: 11.0
2016-10-04 	 actual: 1 0 :prediction 	 result: -22.0
2016-10-03 	 actual: 1 1 :prediction 	 result: 18.0
2016-09-30 	 actual: 0 1 :prediction 	 result: -4.0
2016-09-29 	 

2016-03-01 	 actual: 0 0 :prediction 	 result: 5.0
2016-02-29 	 actual: 0 0 :prediction 	 result: 14.0
2016-02-26 	 actual: 0 0 :prediction 	 result: 18.0
2016-02-25 	 actual: 1 0 :prediction 	 result: -18.0
2016-02-24 	 actual: 0 0 :prediction 	 result: 34.0
2016-02-23 	 actual: 0 1 :prediction 	 result: -4.0
2016-02-22 	 actual: 1 0 :prediction 	 result: -5.0
2016-02-19 	 actual: 0 1 :prediction 	 result: -4.0
2016-02-18 	 actual: 0 1 :prediction 	 result: -3.0
2016-02-17 	 actual: 1 0 :prediction 	 result: -61.0
2016-02-16 	 actual: 0 0 :prediction 	 result: 11.0
2016-02-15 	 actual: 0 0 :prediction 	 result: 11.0
2016-02-12 	 actual: 0 0 :prediction 	 result: 2.0
2016-02-11 	 actual: 1 0 :prediction 	 result: -8.0
2016-02-10 	 actual: 0 0 :prediction 	 result: 16.0
2016-02-09 	 actual: 1 0 :prediction 	 result: -13.0
2016-02-08 	 actual: 0 0 :prediction 	 result: 24.0
2016-02-05 	 actual: 0 1 :prediction 	 result: -0.0
2016-02-04 	 actual: 1 0 :prediction 	 result: -29.0
2016-02-03

...

2014-09-23 	 actual: 0 1 :prediction 	 result: -34.0
2014-09-22 	 actual: 0 1 :prediction 	 result: -4.0
2014-09-19 	 actual: 1 1 :prediction 	 result: 30.0
2014-09-18 	 actual: 0 0 :prediction 	 result: 19.0
2014-09-17 	 actual: 1 0 :prediction 	 result: -3.0
2014-09-16 	 actual: 1 0 :prediction 	 result: -18.0
2014-09-15 	 actual: 1 0 :prediction 	 result: -11.0
2014-09-12 	 actual: 1 1 :prediction 	 result: 3.0
2014-09-11 	 actual: 0 0 :prediction 	 result: 39.0
2014-09-10 	 actual: 0 0 :prediction 	 result: 10.0
2014-09-09 	 actual: 1 0 :prediction 	 result: -4.0
2014-09-08 	 actual: 1 0 :prediction 	 result: -4.0
2014-09-05 	 actual: 1 0 :prediction 	 result: -7.0
2014-09-04 	 actual: 1 0 :prediction 	 result: -30.0
2014-09-03 	 actual: 1 0 :prediction 	 result: -56.0
2014-09-02 	 actual: 1 0 :prediction 	 result: -8.0
2014-09-01 	 actual: 1 0 :prediction 	 result: -10.0
2014-08-29 	 actual: 1 0 :prediction 	 result: -8.0
2014-08-28 	 actual: 0 0 :prediction 	 result: 49.0
2014-08

2013-12-04 	 actual: 0 0 :prediction 	 result: 23.0
2013-12-03 	 actual: 0 1 :prediction 	 result: -31.0
2013-12-02 	 actual: 0 0 :prediction 	 result: 26.0
2013-11-29 	 actual: 1 0 :prediction 	 result: -17.0
2013-11-28 	 actual: 0 0 :prediction 	 result: 30.0
2013-11-27 	 actual: 1 0 :prediction 	 result: -25.0
2013-11-26 	 actual: 0 0 :prediction 	 result: 27.0
2013-11-25 	 actual: 0 0 :prediction 	 result: 10.0
2013-11-22 	 actual: 0 0 :prediction 	 result: 5.0
2013-11-21 	 actual: 1 0 :prediction 	 result: -2.0
2013-11-20 	 actual: 0 0 :prediction 	 result: 10.0
2013-11-19 	 actual: 0 0 :prediction 	 result: 5.0
2013-11-18 	 actual: 1 0 :prediction 	 result: -73.0
2013-11-15 	 actual: 1 0 :prediction 	 result: -38.0
2013-11-14 	 actual: 1 0 :prediction 	 result: -5.0
2013-11-13 	 actual: 0 0 :prediction 	 result: 30.0
2013-11-12 	 actual: 0 0 :prediction 	 result: 23.0
2013-11-08 	 actual: 1 0 :prediction 	 result: -1.0
2013-11-07 	 actual: 0 0 :prediction 	 result: 12.0
2013-11-0

2013-01-15 	 actual: 0 0 :prediction 	 result: 18.0
2013-01-14 	 actual: 0 1 :prediction 	 result: -5.0
2013-01-11 	 actual: 0 0 :prediction 	 result: 17.0
2013-01-10 	 actual: 0 0 :prediction 	 result: 9.0
2013-01-09 	 actual: 0 0 :prediction 	 result: 2.0
2013-01-08 	 actual: 0 0 :prediction 	 result: 8.0
2013-01-07 	 actual: 0 0 :prediction 	 result: 22.0
2013-01-04 	 actual: 0 0 :prediction 	 result: 24.0
2013-01-03 	 actual: 1 0 :prediction 	 result: -5.0
2013-01-02 	 actual: 0 0 :prediction 	 result: 8.0
2012-12-28 	 actual: 0 1 :prediction 	 result: -20.0
2012-12-27 	 actual: 1 0 :prediction 	 result: -19.0
2012-12-21 	 actual: 0 0 :prediction 	 result: 3.0
2012-12-20 	 actual: 1 1 :prediction 	 result: 5.0
2012-12-19 	 actual: 1 1 :prediction 	 result: 21.0
2012-12-18 	 actual: 1 0 :prediction 	 result: -17.0
2012-12-17 	 actual: 1 0 :prediction 	 result: -3.0
2012-12-14 	 actual: 1 1 :prediction 	 result: 17.0
2012-12-13 	 actual: 0 0 :prediction 	 result: 5.0
2012-12-12 	 act

2012-02-20 	 actual: 0 0 :prediction 	 result: 17.0
2012-02-17 	 actual: 1 0 :prediction 	 result: -15.0
2012-02-16 	 actual: 1 0 :prediction 	 result: -2.0
2012-02-15 	 actual: 0 0 :prediction 	 result: 18.0
2012-02-14 	 actual: 0 1 :prediction 	 result: -6.0
2012-02-13 	 actual: 0 0 :prediction 	 result: 2.0
2012-02-10 	 actual: 0 0 :prediction 	 result: 26.0
2012-02-09 	 actual: 0 0 :prediction 	 result: 23.0
2012-02-08 	 actual: 0 0 :prediction 	 result: 14.0
2012-02-07 	 actual: 0 0 :prediction 	 result: 19.0
2012-02-06 	 actual: 1 0 :prediction 	 result: -16.0
2012-02-03 	 actual: 1 0 :prediction 	 result: -14.0
2012-02-02 	 actual: 0 0 :prediction 	 result: 7.0
2012-02-01 	 actual: 1 0 :prediction 	 result: -42.0
2012-01-31 	 actual: 1 0 :prediction 	 result: -5.0
2012-01-30 	 actual: 1 1 :prediction 	 result: 14.0
2012-01-27 	 actual: 1 0 :prediction 	 result: -5.0
2012-01-26 	 actual: 1 0 :prediction 	 result: -41.0
2012-01-25 	 actual: 0 0 :prediction 	 result: 14.0
2012-01-2

2011-03-30 	 actual: 0 0 :prediction 	 result: 18.0
2011-03-29 	 actual: 1 0 :prediction 	 result: -4.0
2011-03-28 	 actual: 1 0 :prediction 	 result: -12.0
2011-03-25 	 actual: 1 0 :prediction 	 result: -29.0
2011-03-24 	 actual: 1 0 :prediction 	 result: -16.0
2011-03-23 	 actual: 1 0 :prediction 	 result: -5.0
2011-03-22 	 actual: 0 1 :prediction 	 result: -29.0
2011-03-21 	 actual: 1 0 :prediction 	 result: -12.0
2011-03-18 	 actual: 0 0 :prediction 	 result: 1.0
2011-03-17 	 actual: 1 0 :prediction 	 result: -30.0
2011-03-16 	 actual: 0 0 :prediction 	 result: 22.0
2011-03-15 	 actual: 1 0 :prediction 	 result: -13.0
2011-03-14 	 actual: 1 0 :prediction 	 result: -8.0
2011-03-11 	 actual: 0 0 :prediction 	 result: 7.0
2011-03-10 	 actual: 0 0 :prediction 	 result: 16.0
2011-03-09 	 actual: 0 0 :prediction 	 result: 9.0
2011-03-08 	 actual: 0 0 :prediction 	 result: 11.0
2011-03-07 	 actual: 1 0 :prediction 	 result: -20.0
2011-03-04 	 actual: 1 0 :prediction 	 result: -11.0
2011-0

### 3. K-Nearest Neighbors

After some optimization, the parameter "number of neighbors" was set at 200.

In [17]:
train_period = 1000
test_period = 2000
date, predictions, y_actual, pred_proba, results = [], [], [], [], []
model = KNeighborsClassifier(n_neighbors = 200)

for t in range(test_period):
    X_train = data_fw20_all.iloc[(-train_period - t - 1) : (-t - 1)][['Close_ret_-1', 'Volume_ret_-1', 
                                                                      'OpenInt_ret_-1']]
    y_train = data_fw20_all.iloc[(-train_period - t - 1) : (-t - 1)][['FW20_dir_o_c']]
    X_test = data_fw20_all.iloc[-t - 1][['Close_ret_-1', 'Volume_ret_-1', 'OpenInt_ret_-1']]
    y_test = data_fw20_all.iloc[-t - 1]['FW20_dir_o_c']

    model.fit(X_train, y_train)
    prediction = model.predict(np.array(X_test).reshape(1,-1))
    prediction_proba = model.predict_proba(np.array(X_test).reshape(1,-1))
    date.append(np.datetime64(data_fw20_all.index[-t -1], 'D'))
    y_actual.append(int(y_test))
    predictions.append(prediction[0])
    pred_proba.append(prediction_proba)
    
    result = abs(data_fw20_all.iloc[-t - 1]['Close'] - data_fw20_all.iloc[-t - 1]['Open'])
    results.append(result if y_test == prediction else -result)

accuracy = 0
for i in range(len(predictions)):
    if y_actual[i] == predictions[i]:
        accuracy+=1
print('Model summary')
print('Accuracy: {}'.format(round(accuracy / len(predictions),4)))
print('Result (pts.): {}'.format(sum(results)))
print('Average result per 1 session: {}'.format(sum(results) / test_period))
print('*'*100)

for i in range(len(predictions)):
    print(date[i], '\t', 'actual:', y_actual[i], predictions[i], ':prediction', 'preds_proba:', pred_proba[i],
         '\t', 'result:', results[i])

Model summary
Accuracy: 0.527
Result (pts.): 2488.0
Average result per 1 session: 1.244
****************************************************************************************************
2018-12-28 	 actual: 1 0 :prediction preds_proba: [[0.555 0.445]] 	 result: -16.0
2018-12-27 	 actual: 0 0 :prediction preds_proba: [[0.54 0.46]] 	 result: 40.0
2018-12-21 	 actual: 0 0 :prediction preds_proba: [[0.54 0.46]] 	 result: 38.0
2018-12-20 	 actual: 0 0 :prediction preds_proba: [[0.535 0.465]] 	 result: 10.0
2018-12-19 	 actual: 1 0 :prediction preds_proba: [[0.55 0.45]] 	 result: -33.0
2018-12-18 	 actual: 1 0 :prediction preds_proba: [[0.555 0.445]] 	 result: -46.0
2018-12-17 	 actual: 0 0 :prediction preds_proba: [[0.52 0.48]] 	 result: 22.0
2018-12-14 	 actual: 1 0 :prediction preds_proba: [[0.53 0.47]] 	 result: -13.0
2018-12-13 	 actual: 1 0 :prediction preds_proba: [[0.525 0.475]] 	 result: -21.0
2018-12-12 	 actual: 1 0 :prediction preds_proba: [[0.55 0.45]] 	 result: -53.0
2018-12

2018-04-25 	 actual: 0 1 :prediction preds_proba: [[0.455 0.545]] 	 result: -7.0
2018-04-24 	 actual: 0 0 :prediction preds_proba: [[0.51 0.49]] 	 result: 25.0
2018-04-23 	 actual: 0 0 :prediction preds_proba: [[0.54 0.46]] 	 result: 4.0
2018-04-20 	 actual: 0 0 :prediction preds_proba: [[0.505 0.495]] 	 result: 17.0
2018-04-19 	 actual: 1 0 :prediction preds_proba: [[0.505 0.495]] 	 result: -12.0
2018-04-18 	 actual: 0 1 :prediction preds_proba: [[0.495 0.505]] 	 result: -2.0
2018-04-17 	 actual: 0 0 :prediction preds_proba: [[0.52 0.48]] 	 result: 0.0
2018-04-16 	 actual: 0 0 :prediction preds_proba: [[0.545 0.455]] 	 result: 17.0
2018-04-13 	 actual: 0 1 :prediction preds_proba: [[0.475 0.525]] 	 result: -6.0
2018-04-12 	 actual: 1 0 :prediction preds_proba: [[0.51 0.49]] 	 result: -41.0
2018-04-11 	 actual: 0 0 :prediction preds_proba: [[0.515 0.485]] 	 result: 4.0
2018-04-10 	 actual: 0 1 :prediction preds_proba: [[0.47 0.53]] 	 result: -5.0
2018-04-09 	 actual: 0 0 :prediction pr

2017-10-30 	 actual: 1 1 :prediction preds_proba: [[0.465 0.535]] 	 result: 23.0
2017-10-27 	 actual: 1 1 :prediction preds_proba: [[0.49 0.51]] 	 result: 23.0
2017-10-26 	 actual: 1 0 :prediction preds_proba: [[0.535 0.465]] 	 result: -5.0
2017-10-25 	 actual: 1 1 :prediction preds_proba: [[0.485 0.515]] 	 result: 3.0
2017-10-24 	 actual: 0 0 :prediction preds_proba: [[0.54 0.46]] 	 result: 27.0
2017-10-23 	 actual: 1 0 :prediction preds_proba: [[0.51 0.49]] 	 result: -9.0
2017-10-20 	 actual: 0 0 :prediction preds_proba: [[0.51 0.49]] 	 result: 22.0
2017-10-19 	 actual: 0 0 :prediction preds_proba: [[0.515 0.485]] 	 result: 31.0
2017-10-18 	 actual: 0 0 :prediction preds_proba: [[0.55 0.45]] 	 result: 16.0
2017-10-17 	 actual: 0 0 :prediction preds_proba: [[0.53 0.47]] 	 result: 19.0
2017-10-16 	 actual: 0 0 :prediction preds_proba: [[0.535 0.465]] 	 result: 6.0
2017-10-13 	 actual: 0 0 :prediction preds_proba: [[0.55 0.45]] 	 result: 9.0
2017-10-12 	 actual: 0 1 :prediction preds_pr

2017-05-30 	 actual: 0 0 :prediction preds_proba: [[0.535 0.465]] 	 result: 22.0
2017-05-29 	 actual: 0 0 :prediction preds_proba: [[0.54 0.46]] 	 result: 4.0
2017-05-26 	 actual: 0 0 :prediction preds_proba: [[0.545 0.455]] 	 result: 26.0
2017-05-25 	 actual: 1 0 :prediction preds_proba: [[0.575 0.425]] 	 result: -6.0
2017-05-24 	 actual: 1 0 :prediction preds_proba: [[0.575 0.425]] 	 result: -25.0
2017-05-23 	 actual: 0 0 :prediction preds_proba: [[0.57 0.43]] 	 result: 30.0
2017-05-22 	 actual: 0 0 :prediction preds_proba: [[0.565 0.435]] 	 result: 1.0
2017-05-19 	 actual: 1 1 :prediction preds_proba: [[0.495 0.505]] 	 result: 18.0
2017-05-18 	 actual: 0 0 :prediction preds_proba: [[0.545 0.455]] 	 result: 6.0
2017-05-17 	 actual: 0 1 :prediction preds_proba: [[0.49 0.51]] 	 result: -14.0
2017-05-16 	 actual: 0 0 :prediction preds_proba: [[0.555 0.445]] 	 result: 48.0
2017-05-15 	 actual: 0 0 :prediction preds_proba: [[0.535 0.465]] 	 result: 14.0
2017-05-12 	 actual: 0 0 :predictio

2016-11-09 	 actual: 1 0 :prediction preds_proba: [[0.535 0.465]] 	 result: -43.0
2016-11-08 	 actual: 1 0 :prediction preds_proba: [[0.505 0.495]] 	 result: -17.0
2016-11-07 	 actual: 0 0 :prediction preds_proba: [[0.525 0.475]] 	 result: 12.0
2016-11-04 	 actual: 0 0 :prediction preds_proba: [[0.535 0.465]] 	 result: 1.0
2016-11-03 	 actual: 1 1 :prediction preds_proba: [[0.49 0.51]] 	 result: 3.0
2016-11-02 	 actual: 0 0 :prediction preds_proba: [[0.58 0.42]] 	 result: 39.0
2016-10-31 	 actual: 0 0 :prediction preds_proba: [[0.555 0.445]] 	 result: 5.0
2016-10-28 	 actual: 1 1 :prediction preds_proba: [[0.49 0.51]] 	 result: 28.0
2016-10-27 	 actual: 1 0 :prediction preds_proba: [[0.555 0.445]] 	 result: -12.0
2016-10-26 	 actual: 0 1 :prediction preds_proba: [[0.485 0.515]] 	 result: -2.0
2016-10-25 	 actual: 1 1 :prediction preds_proba: [[0.49 0.51]] 	 result: 2.0
2016-10-24 	 actual: 1 0 :prediction preds_proba: [[0.58 0.42]] 	 result: -27.0
2016-10-21 	 actual: 1 0 :prediction p

...

2013-08-01 	 actual: 1 0 :prediction preds_proba: [[0.53 0.47]] 	 result: -8.0
2013-07-31 	 actual: 1 0 :prediction preds_proba: [[0.58 0.42]] 	 result: -43.0
2013-07-30 	 actual: 1 0 :prediction preds_proba: [[0.55 0.45]] 	 result: -4.0
2013-07-29 	 actual: 1 0 :prediction preds_proba: [[0.54 0.46]] 	 result: -8.0
2013-07-26 	 actual: 0 0 :prediction preds_proba: [[0.59 0.41]] 	 result: 17.0
2013-07-25 	 actual: 0 0 :prediction preds_proba: [[0.565 0.435]] 	 result: 7.0
2013-07-24 	 actual: 1 0 :prediction preds_proba: [[0.58 0.42]] 	 result: -7.0
2013-07-23 	 actual: 0 0 :prediction preds_proba: [[0.51 0.49]] 	 result: 16.0
2013-07-22 	 actual: 1 0 :prediction preds_proba: [[0.52 0.48]] 	 result: -1.0
2013-07-19 	 actual: 1 0 :prediction preds_proba: [[0.505 0.495]] 	 result: -18.0
2013-07-18 	 actual: 0 0 :prediction preds_proba: [[0.545 0.455]] 	 result: 12.0
2013-07-17 	 actual: 1 0 :prediction preds_proba: [[0.545 0.455]] 	 result: -16.0
2013-07-16 	 actual: 0 0 :prediction preds

2013-01-14 	 actual: 0 1 :prediction preds_proba: [[0.48 0.52]] 	 result: -5.0
2013-01-11 	 actual: 0 1 :prediction preds_proba: [[0.49 0.51]] 	 result: -17.0
2013-01-10 	 actual: 0 0 :prediction preds_proba: [[0.515 0.485]] 	 result: 9.0
2013-01-09 	 actual: 0 0 :prediction preds_proba: [[0.5 0.5]] 	 result: 2.0
2013-01-08 	 actual: 0 1 :prediction preds_proba: [[0.49 0.51]] 	 result: -8.0
2013-01-07 	 actual: 0 0 :prediction preds_proba: [[0.5 0.5]] 	 result: 22.0
2013-01-04 	 actual: 0 0 :prediction preds_proba: [[0.54 0.46]] 	 result: 24.0
2013-01-03 	 actual: 1 0 :prediction preds_proba: [[0.52 0.48]] 	 result: -5.0
2013-01-02 	 actual: 0 1 :prediction preds_proba: [[0.495 0.505]] 	 result: -8.0
2012-12-28 	 actual: 0 1 :prediction preds_proba: [[0.46 0.54]] 	 result: -20.0
2012-12-27 	 actual: 1 0 :prediction preds_proba: [[0.565 0.435]] 	 result: -19.0
2012-12-21 	 actual: 0 0 :prediction preds_proba: [[0.515 0.485]] 	 result: 3.0
2012-12-20 	 actual: 1 1 :prediction preds_proba

2012-06-25 	 actual: 0 0 :prediction preds_proba: [[0.565 0.435]] 	 result: 12.0
2012-06-22 	 actual: 1 0 :prediction preds_proba: [[0.585 0.415]] 	 result: -12.0
2012-06-21 	 actual: 0 0 :prediction preds_proba: [[0.525 0.475]] 	 result: 3.0
2012-06-20 	 actual: 0 0 :prediction preds_proba: [[0.65 0.35]] 	 result: 15.0
2012-06-19 	 actual: 1 1 :prediction preds_proba: [[0.48 0.52]] 	 result: 56.0
2012-06-18 	 actual: 0 0 :prediction preds_proba: [[0.54 0.46]] 	 result: 24.0
2012-06-15 	 actual: 1 0 :prediction preds_proba: [[0.585 0.415]] 	 result: -23.0
2012-06-14 	 actual: 0 0 :prediction preds_proba: [[0.515 0.485]] 	 result: 11.0
2012-06-13 	 actual: 0 0 :prediction preds_proba: [[0.52 0.48]] 	 result: 8.0
2012-06-12 	 actual: 1 0 :prediction preds_proba: [[0.53 0.47]] 	 result: -51.0
2012-06-11 	 actual: 1 0 :prediction preds_proba: [[0.53 0.47]] 	 result: -4.0
2012-06-08 	 actual: 1 0 :prediction preds_proba: [[0.52 0.48]] 	 result: -69.0
2012-06-06 	 actual: 1 0 :prediction pre

2011-12-07 	 actual: 0 0 :prediction preds_proba: [[0.55 0.45]] 	 result: 5.0
2011-12-06 	 actual: 0 0 :prediction preds_proba: [[0.535 0.465]] 	 result: 27.0
2011-12-05 	 actual: 1 0 :prediction preds_proba: [[0.55 0.45]] 	 result: -10.0
2011-12-02 	 actual: 0 1 :prediction preds_proba: [[0.48 0.52]] 	 result: -15.0
2011-12-01 	 actual: 0 0 :prediction preds_proba: [[0.54 0.46]] 	 result: 22.0
2011-11-30 	 actual: 1 0 :prediction preds_proba: [[0.58 0.42]] 	 result: -115.0
2011-11-29 	 actual: 0 1 :prediction preds_proba: [[0.495 0.505]] 	 result: -20.0
2011-11-28 	 actual: 1 0 :prediction preds_proba: [[0.56 0.44]] 	 result: -17.0
2011-11-25 	 actual: 1 1 :prediction preds_proba: [[0.495 0.505]] 	 result: 29.0
2011-11-24 	 actual: 0 1 :prediction preds_proba: [[0.495 0.505]] 	 result: -0.0
2011-11-23 	 actual: 0 0 :prediction preds_proba: [[0.505 0.495]] 	 result: 26.0
2011-11-22 	 actual: 0 0 :prediction preds_proba: [[0.64 0.36]] 	 result: 3.0
2011-11-21 	 actual: 0 1 :prediction p

2011-05-20 	 actual: 0 0 :prediction preds_proba: [[0.61 0.39]] 	 result: 4.0
2011-05-19 	 actual: 1 0 :prediction preds_proba: [[0.545 0.455]] 	 result: -13.0
2011-05-18 	 actual: 1 0 :prediction preds_proba: [[0.6 0.4]] 	 result: -7.0
2011-05-17 	 actual: 0 0 :prediction preds_proba: [[0.51 0.49]] 	 result: 22.0
2011-05-16 	 actual: 1 1 :prediction preds_proba: [[0.485 0.515]] 	 result: 30.0
2011-05-13 	 actual: 0 0 :prediction preds_proba: [[0.5 0.5]] 	 result: 20.0
2011-05-12 	 actual: 0 0 :prediction preds_proba: [[0.58 0.42]] 	 result: 13.0
2011-05-11 	 actual: 0 0 :prediction preds_proba: [[0.61 0.39]] 	 result: 14.0
2011-05-10 	 actual: 1 1 :prediction preds_proba: [[0.495 0.505]] 	 result: 14.0
2011-05-09 	 actual: 0 0 :prediction preds_proba: [[0.54 0.46]] 	 result: 13.0
2011-05-06 	 actual: 1 0 :prediction preds_proba: [[0.59 0.41]] 	 result: -5.0
2011-05-05 	 actual: 0 0 :prediction preds_proba: [[0.515 0.485]] 	 result: 18.0
2011-05-04 	 actual: 0 0 :prediction preds_proba

### 4. Random Forest Classifier

Number of trees in each forest set at 100, max depth of each tree set at 3.

In [18]:
train_period = 1000
test_period = 2000
date, predictions, y_actual, pred_proba, results = [], [], [], [], []
model = RandomForestClassifier(n_estimators=100, max_depth=3)

for t in range(test_period):
    X_train = data_fw20_all.iloc[(-train_period - t - 1) : (-t - 1)][['Close_ret_-1', 'Volume_ret_-1', 
                                                                      'OpenInt_ret_-1']]
    y_train = data_fw20_all.iloc[(-train_period - t - 1) : (-t - 1)][['FW20_dir_o_c']]
    X_test = data_fw20_all.iloc[-t - 1][['Close_ret_-1', 'Volume_ret_-1', 'OpenInt_ret_-1']]
    y_test = data_fw20_all.iloc[-t - 1]['FW20_dir_o_c']

    model.fit(X_train, y_train)
    prediction = model.predict(np.array(X_test).reshape(1,-1))
    prediction_proba = model.predict_proba(np.array(X_test).reshape(1,-1))
    date.append(np.datetime64(data_fw20_all.index[-t -1], 'D'))
    y_actual.append(int(y_test))
    predictions.append(prediction[0])
    pred_proba.append(prediction_proba)
    
    result = abs(data_fw20_all.iloc[-t - 1]['Close'] - data_fw20_all.iloc[-t - 1]['Open'])
    results.append(result if y_test == prediction else -result)

accuracy = 0
for i in range(len(predictions)):
    if y_actual[i] == predictions[i]:
        accuracy+=1
print('Model summary')
print('Accuracy: {}'.format(round(accuracy / len(predictions),4)))
print('Result (pts.): {}'.format(sum(results)))
print('Average result per 1 session: {}'.format(sum(results) / test_period))
print('*'*100)

for i in range(len(predictions)):
    print(date[i], '\t', 'actual:', y_actual[i], predictions[i], ':prediction', 'preds_proba:', pred_proba[i],
         '\t', 'result:', results[i])

Model summary
Accuracy: 0.5385
Result (pts.): 2928.0
Average result per 1 session: 1.464
****************************************************************************************************
2018-12-28 	 actual: 1 0 :prediction preds_proba: [[0.50044503 0.49955497]] 	 result: -16.0
2018-12-27 	 actual: 0 0 :prediction preds_proba: [[0.51363691 0.48636309]] 	 result: 40.0
2018-12-21 	 actual: 0 1 :prediction preds_proba: [[0.40106096 0.59893904]] 	 result: -38.0
2018-12-20 	 actual: 0 0 :prediction preds_proba: [[0.52151162 0.47848838]] 	 result: 10.0
2018-12-19 	 actual: 1 0 :prediction preds_proba: [[0.58891741 0.41108259]] 	 result: -33.0
2018-12-18 	 actual: 1 0 :prediction preds_proba: [[0.50354899 0.49645101]] 	 result: -46.0
2018-12-17 	 actual: 0 0 :prediction preds_proba: [[0.51417615 0.48582385]] 	 result: 22.0
2018-12-14 	 actual: 1 0 :prediction preds_proba: [[0.57470133 0.42529867]] 	 result: -13.0
2018-12-13 	 actual: 1 0 :prediction preds_proba: [[0.53881355 0.46118645]] 	

2018-04-27 	 actual: 0 0 :prediction preds_proba: [[0.5498349 0.4501651]] 	 result: 12.0
2018-04-26 	 actual: 1 0 :prediction preds_proba: [[0.5251855 0.4748145]] 	 result: -18.0
2018-04-25 	 actual: 0 1 :prediction preds_proba: [[0.44725936 0.55274064]] 	 result: -7.0
2018-04-24 	 actual: 0 0 :prediction preds_proba: [[0.55320718 0.44679282]] 	 result: 25.0
2018-04-23 	 actual: 0 1 :prediction preds_proba: [[0.44928811 0.55071189]] 	 result: -4.0
2018-04-20 	 actual: 0 1 :prediction preds_proba: [[0.49408259 0.50591741]] 	 result: -17.0
2018-04-19 	 actual: 1 0 :prediction preds_proba: [[0.55259759 0.44740241]] 	 result: -12.0
2018-04-18 	 actual: 0 0 :prediction preds_proba: [[0.54264398 0.45735602]] 	 result: 2.0
2018-04-17 	 actual: 0 0 :prediction preds_proba: [[0.57808136 0.42191864]] 	 result: 0.0
2018-04-16 	 actual: 0 0 :prediction preds_proba: [[0.50058428 0.49941572]] 	 result: 17.0
2018-04-13 	 actual: 0 0 :prediction preds_proba: [[0.55532506 0.44467494]] 	 result: 6.0
201

2017-11-23 	 actual: 1 1 :prediction preds_proba: [[0.48880974 0.51119026]] 	 result: 29.0
2017-11-22 	 actual: 0 0 :prediction preds_proba: [[0.53607595 0.46392405]] 	 result: 8.0
2017-11-21 	 actual: 1 0 :prediction preds_proba: [[0.56357538 0.43642462]] 	 result: -15.0
2017-11-20 	 actual: 1 0 :prediction preds_proba: [[0.58716139 0.41283861]] 	 result: -34.0
2017-11-17 	 actual: 1 0 :prediction preds_proba: [[0.58028024 0.41971976]] 	 result: -26.0
2017-11-16 	 actual: 0 1 :prediction preds_proba: [[0.45616357 0.54383643]] 	 result: -17.0
2017-11-15 	 actual: 0 1 :prediction preds_proba: [[0.4722129 0.5277871]] 	 result: -38.0
2017-11-14 	 actual: 1 0 :prediction preds_proba: [[0.55965209 0.44034791]] 	 result: -5.0
2017-11-13 	 actual: 0 0 :prediction preds_proba: [[0.54108503 0.45891497]] 	 result: 0.0
2017-11-10 	 actual: 0 1 :prediction preds_proba: [[0.41537871 0.58462129]] 	 result: -18.0
2017-11-09 	 actual: 0 1 :prediction preds_proba: [[0.43318092 0.56681908]] 	 result: -3

2017-05-31 	 actual: 0 0 :prediction preds_proba: [[0.54274785 0.45725215]] 	 result: 10.0
2017-05-30 	 actual: 0 0 :prediction preds_proba: [[0.64282581 0.35717419]] 	 result: 22.0
2017-05-29 	 actual: 0 1 :prediction preds_proba: [[0.42081475 0.57918525]] 	 result: -4.0
2017-05-26 	 actual: 0 0 :prediction preds_proba: [[0.59074597 0.40925403]] 	 result: 26.0
2017-05-25 	 actual: 1 0 :prediction preds_proba: [[0.58876052 0.41123948]] 	 result: -6.0
2017-05-24 	 actual: 1 0 :prediction preds_proba: [[0.55482948 0.44517052]] 	 result: -25.0
2017-05-23 	 actual: 0 1 :prediction preds_proba: [[0.44646596 0.55353404]] 	 result: -30.0
2017-05-22 	 actual: 0 1 :prediction preds_proba: [[0.47478594 0.52521406]] 	 result: -1.0
2017-05-19 	 actual: 1 1 :prediction preds_proba: [[0.43291401 0.56708599]] 	 result: 18.0
2017-05-18 	 actual: 0 1 :prediction preds_proba: [[0.42278642 0.57721358]] 	 result: -6.0
2017-05-17 	 actual: 0 0 :prediction preds_proba: [[0.5089162 0.4910838]] 	 result: 14.0

2016-11-14 	 actual: 0 0 :prediction preds_proba: [[0.55747163 0.44252837]] 	 result: 51.0
2016-11-10 	 actual: 0 0 :prediction preds_proba: [[0.53323343 0.46676657]] 	 result: 14.0
2016-11-09 	 actual: 1 1 :prediction preds_proba: [[0.48238502 0.51761498]] 	 result: 43.0
2016-11-08 	 actual: 1 0 :prediction preds_proba: [[0.52547463 0.47452537]] 	 result: -17.0
2016-11-07 	 actual: 0 0 :prediction preds_proba: [[0.60006086 0.39993914]] 	 result: 12.0
2016-11-04 	 actual: 0 0 :prediction preds_proba: [[0.60274167 0.39725833]] 	 result: 1.0
2016-11-03 	 actual: 1 1 :prediction preds_proba: [[0.479791 0.520209]] 	 result: 3.0
2016-11-02 	 actual: 0 0 :prediction preds_proba: [[0.58980886 0.41019114]] 	 result: 39.0
2016-10-31 	 actual: 0 0 :prediction preds_proba: [[0.56851682 0.43148318]] 	 result: 5.0
2016-10-28 	 actual: 1 0 :prediction preds_proba: [[0.52557249 0.47442751]] 	 result: -28.0
2016-10-27 	 actual: 1 0 :prediction preds_proba: [[0.58569357 0.41430643]] 	 result: -12.0
201

...

2013-08-02 	 actual: 1 0 :prediction preds_proba: [[0.53158377 0.46841623]] 	 result: -51.0
2013-08-01 	 actual: 1 0 :prediction preds_proba: [[0.5673302 0.4326698]] 	 result: -8.0
2013-07-31 	 actual: 1 0 :prediction preds_proba: [[0.56200879 0.43799121]] 	 result: -43.0
2013-07-30 	 actual: 1 0 :prediction preds_proba: [[0.5659338 0.4340662]] 	 result: -4.0
2013-07-29 	 actual: 1 0 :prediction preds_proba: [[0.50193369 0.49806631]] 	 result: -8.0
2013-07-26 	 actual: 0 0 :prediction preds_proba: [[0.51689448 0.48310552]] 	 result: 17.0
2013-07-25 	 actual: 0 0 :prediction preds_proba: [[0.5606681 0.4393319]] 	 result: 7.0
2013-07-24 	 actual: 1 0 :prediction preds_proba: [[0.54197279 0.45802721]] 	 result: -7.0
2013-07-23 	 actual: 0 1 :prediction preds_proba: [[0.47593831 0.52406169]] 	 result: -16.0
2013-07-22 	 actual: 1 0 :prediction preds_proba: [[0.56383844 0.43616156]] 	 result: -1.0
2013-07-19 	 actual: 1 0 :prediction preds_proba: [[0.53645798 0.46354202]] 	 result: -18.0
20

2013-01-16 	 actual: 1 0 :prediction preds_proba: [[0.52475669 0.47524331]] 	 result: -38.0
2013-01-15 	 actual: 0 0 :prediction preds_proba: [[0.52033156 0.47966844]] 	 result: 18.0
2013-01-14 	 actual: 0 1 :prediction preds_proba: [[0.37068397 0.62931603]] 	 result: -5.0
2013-01-11 	 actual: 0 0 :prediction preds_proba: [[0.5447121 0.4552879]] 	 result: 17.0
2013-01-10 	 actual: 0 0 :prediction preds_proba: [[0.53234291 0.46765709]] 	 result: 9.0
2013-01-09 	 actual: 0 0 :prediction preds_proba: [[0.51568922 0.48431078]] 	 result: 2.0
2013-01-08 	 actual: 0 0 :prediction preds_proba: [[0.51954124 0.48045876]] 	 result: 8.0
2013-01-07 	 actual: 0 0 :prediction preds_proba: [[0.52537119 0.47462881]] 	 result: 22.0
2013-01-04 	 actual: 0 0 :prediction preds_proba: [[0.53399884 0.46600116]] 	 result: 24.0
2013-01-03 	 actual: 1 0 :prediction preds_proba: [[0.56925907 0.43074093]] 	 result: -5.0
2013-01-02 	 actual: 0 0 :prediction preds_proba: [[0.51228114 0.48771886]] 	 result: 8.0
2012

2012-06-27 	 actual: 1 0 :prediction preds_proba: [[0.55456802 0.44543198]] 	 result: -12.0
2012-06-26 	 actual: 1 0 :prediction preds_proba: [[0.54624375 0.45375625]] 	 result: -33.0
2012-06-25 	 actual: 0 0 :prediction preds_proba: [[0.55489351 0.44510649]] 	 result: 12.0
2012-06-22 	 actual: 1 0 :prediction preds_proba: [[0.56984228 0.43015772]] 	 result: -12.0
2012-06-21 	 actual: 0 0 :prediction preds_proba: [[0.53546957 0.46453043]] 	 result: 3.0
2012-06-20 	 actual: 0 0 :prediction preds_proba: [[0.55836987 0.44163013]] 	 result: 15.0
2012-06-19 	 actual: 1 1 :prediction preds_proba: [[0.44921216 0.55078784]] 	 result: 56.0
2012-06-18 	 actual: 0 0 :prediction preds_proba: [[0.56380166 0.43619834]] 	 result: 24.0
2012-06-15 	 actual: 1 0 :prediction preds_proba: [[0.60349156 0.39650844]] 	 result: -23.0
2012-06-14 	 actual: 0 0 :prediction preds_proba: [[0.52354858 0.47645142]] 	 result: 11.0
2012-06-13 	 actual: 0 0 :prediction preds_proba: [[0.52966734 0.47033266]] 	 result: 8

2011-12-08 	 actual: 0 0 :prediction preds_proba: [[0.5460589 0.4539411]] 	 result: 57.0
2011-12-07 	 actual: 0 0 :prediction preds_proba: [[0.5444186 0.4555814]] 	 result: 5.0
2011-12-06 	 actual: 0 0 :prediction preds_proba: [[0.61384098 0.38615902]] 	 result: 27.0
2011-12-05 	 actual: 1 0 :prediction preds_proba: [[0.56122829 0.43877171]] 	 result: -10.0
2011-12-02 	 actual: 0 1 :prediction preds_proba: [[0.48913524 0.51086476]] 	 result: -15.0
2011-12-01 	 actual: 0 0 :prediction preds_proba: [[0.50397959 0.49602041]] 	 result: 22.0
2011-11-30 	 actual: 1 0 :prediction preds_proba: [[0.55976662 0.44023338]] 	 result: -115.0
2011-11-29 	 actual: 0 1 :prediction preds_proba: [[0.47827943 0.52172057]] 	 result: -20.0
2011-11-28 	 actual: 1 0 :prediction preds_proba: [[0.58576067 0.41423933]] 	 result: -17.0
2011-11-25 	 actual: 1 0 :prediction preds_proba: [[0.53291961 0.46708039]] 	 result: -29.0
2011-11-24 	 actual: 0 0 :prediction preds_proba: [[0.53663052 0.46336948]] 	 result: 0.

2011-05-24 	 actual: 1 0 :prediction preds_proba: [[0.52762524 0.47237476]] 	 result: -8.0
2011-05-23 	 actual: 0 0 :prediction preds_proba: [[0.51234559 0.48765441]] 	 result: 22.0
2011-05-20 	 actual: 0 0 :prediction preds_proba: [[0.56086852 0.43913148]] 	 result: 4.0
2011-05-19 	 actual: 1 0 :prediction preds_proba: [[0.56725721 0.43274279]] 	 result: -13.0
2011-05-18 	 actual: 1 0 :prediction preds_proba: [[0.54811449 0.45188551]] 	 result: -7.0
2011-05-17 	 actual: 0 0 :prediction preds_proba: [[0.56542028 0.43457972]] 	 result: 22.0
2011-05-16 	 actual: 1 1 :prediction preds_proba: [[0.46022042 0.53977958]] 	 result: 30.0
2011-05-13 	 actual: 0 0 :prediction preds_proba: [[0.54466355 0.45533645]] 	 result: 20.0
2011-05-12 	 actual: 0 0 :prediction preds_proba: [[0.56512358 0.43487642]] 	 result: 13.0
2011-05-11 	 actual: 0 0 :prediction preds_proba: [[0.56814038 0.43185962]] 	 result: 14.0
2011-05-10 	 actual: 1 0 :prediction preds_proba: [[0.55109701 0.44890299]] 	 result: -14.

### 5. AdaBoost Classifier (with Decision Tree)

Number of estimators set at 50, max depth of each tree set at 1 ("decision stump").

In [20]:
train_period = 1000
test_period = 2000
date, predictions, y_actual, pred_proba, results = [], [], [], [], []
model = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=50)

for t in range(test_period):
    X_train = data_fw20_all.iloc[(-train_period - t - 1) : (-t - 1)][['Close_ret_-1', 'Volume_ret_-1', 
                                                                      'OpenInt_ret_-1']]
    y_train = data_fw20_all.iloc[(-train_period - t - 1) : (-t - 1)][['FW20_dir_o_c']]
    X_test = data_fw20_all.iloc[-t - 1][['Close_ret_-1', 'Volume_ret_-1', 'OpenInt_ret_-1']]
    y_test = data_fw20_all.iloc[-t - 1]['FW20_dir_o_c']

    model.fit(X_train, y_train)
    prediction = model.predict(np.array(X_test).reshape(1,-1))
    prediction_proba = model.predict_proba(np.array(X_test).reshape(1,-1))
    date.append(np.datetime64(data_fw20_all.index[-t -1], 'D'))
    y_actual.append(int(y_test))
    predictions.append(prediction[0])
    pred_proba.append(prediction_proba)

    result = abs(data_fw20_all.iloc[-t - 1]['Close'] - data_fw20_all.iloc[-t - 1]['Open'])
    results.append(result if y_test == prediction else -result)

accuracy = 0
for i in range(len(predictions)):
    if y_actual[i] == predictions[i]:
        accuracy+=1
print('Model summary')
print('Accuracy: {}'.format(round(accuracy / len(predictions),4)))
print('Result (pts.): {}'.format(sum(results)))
print('Average result per 1 session: {}'.format(sum(results) / test_period))
print('*'*100)

for i in range(len(predictions)):
    print(date[i], '\t', 'actual:', y_actual[i], predictions[i], ':prediction', 'preds_proba:', pred_proba[i],
         '\t', 'result:', results[i])

Model summary
Accuracy: 0.514
Result (pts.): 316.0
Average result per 1 session: 0.158
****************************************************************************************************
2018-12-28 	 actual: 1 0 :prediction preds_proba: [[0.50277047 0.49722953]] 	 result: -16.0
2018-12-27 	 actual: 0 1 :prediction preds_proba: [[0.49817071 0.50182929]] 	 result: -40.0
2018-12-21 	 actual: 0 1 :prediction preds_proba: [[0.49369794 0.50630206]] 	 result: -38.0
2018-12-20 	 actual: 0 0 :prediction preds_proba: [[0.50196018 0.49803982]] 	 result: 10.0
2018-12-19 	 actual: 1 0 :prediction preds_proba: [[0.50670572 0.49329428]] 	 result: -33.0
2018-12-18 	 actual: 1 1 :prediction preds_proba: [[0.4991614 0.5008386]] 	 result: 46.0
2018-12-17 	 actual: 0 0 :prediction preds_proba: [[0.50017306 0.49982694]] 	 result: 22.0
2018-12-14 	 actual: 1 0 :prediction preds_proba: [[0.50286864 0.49713136]] 	 result: -13.0
2018-12-13 	 actual: 1 0 :prediction preds_proba: [[0.54600825 0.45399175]] 	 res

2018-05-04 	 actual: 0 0 :prediction preds_proba: [[0.50074757 0.49925243]] 	 result: 45.0
2018-05-02 	 actual: 0 0 :prediction preds_proba: [[0.50189129 0.49810871]] 	 result: 1.0
2018-04-30 	 actual: 1 0 :prediction preds_proba: [[0.50214182 0.49785818]] 	 result: -11.0
2018-04-27 	 actual: 0 0 :prediction preds_proba: [[0.50199844 0.49800156]] 	 result: 12.0
2018-04-26 	 actual: 1 1 :prediction preds_proba: [[0.49987092 0.50012908]] 	 result: 18.0
2018-04-25 	 actual: 0 1 :prediction preds_proba: [[0.49649509 0.50350491]] 	 result: -7.0
2018-04-24 	 actual: 0 0 :prediction preds_proba: [[0.50248697 0.49751303]] 	 result: 25.0
2018-04-23 	 actual: 0 1 :prediction preds_proba: [[0.49977916 0.50022084]] 	 result: -4.0
2018-04-20 	 actual: 0 1 :prediction preds_proba: [[0.49704652 0.50295348]] 	 result: -17.0
2018-04-19 	 actual: 1 0 :prediction preds_proba: [[0.5014043 0.4985957]] 	 result: -12.0
2018-04-18 	 actual: 0 1 :prediction preds_proba: [[0.49830928 0.50169072]] 	 result: -2.0

2017-12-18 	 actual: 1 0 :prediction preds_proba: [[0.50164632 0.49835368]] 	 result: -31.0
2017-12-15 	 actual: 0 0 :prediction preds_proba: [[0.50227213 0.49772787]] 	 result: 8.0
2017-12-14 	 actual: 1 0 :prediction preds_proba: [[0.50437759 0.49562241]] 	 result: -30.0
2017-12-13 	 actual: 0 0 :prediction preds_proba: [[0.50216523 0.49783477]] 	 result: 7.0
2017-12-12 	 actual: 1 1 :prediction preds_proba: [[0.49871505 0.50128495]] 	 result: 21.0
2017-12-11 	 actual: 0 1 :prediction preds_proba: [[0.49896073 0.50103927]] 	 result: -30.0
2017-12-08 	 actual: 0 1 :prediction preds_proba: [[0.49891802 0.50108198]] 	 result: -9.0
2017-12-07 	 actual: 0 0 :prediction preds_proba: [[0.50120523 0.49879477]] 	 result: 1.0
2017-12-06 	 actual: 0 1 :prediction preds_proba: [[0.49821648 0.50178352]] 	 result: -0.0
2017-12-05 	 actual: 0 1 :prediction preds_proba: [[0.49824986 0.50175014]] 	 result: -1.0
2017-12-04 	 actual: 0 1 :prediction preds_proba: [[0.49573175 0.50426825]] 	 result: -0.0

2017-06-06 	 actual: 1 0 :prediction preds_proba: [[0.50254004 0.49745996]] 	 result: -45.0
2017-06-05 	 actual: 0 1 :prediction preds_proba: [[0.49789052 0.50210948]] 	 result: -31.0
2017-06-02 	 actual: 0 1 :prediction preds_proba: [[0.49837548 0.50162452]] 	 result: -7.0
2017-06-01 	 actual: 0 1 :prediction preds_proba: [[0.49779607 0.50220393]] 	 result: -10.0
2017-05-31 	 actual: 0 1 :prediction preds_proba: [[0.49767368 0.50232632]] 	 result: -10.0
2017-05-30 	 actual: 0 1 :prediction preds_proba: [[0.49656423 0.50343577]] 	 result: -22.0
2017-05-29 	 actual: 0 1 :prediction preds_proba: [[0.49787774 0.50212226]] 	 result: -4.0
2017-05-26 	 actual: 0 0 :prediction preds_proba: [[0.50301994 0.49698006]] 	 result: 26.0
2017-05-25 	 actual: 1 0 :prediction preds_proba: [[0.50416596 0.49583404]] 	 result: -6.0
2017-05-24 	 actual: 1 0 :prediction preds_proba: [[0.5030544 0.4969456]] 	 result: -25.0
2017-05-23 	 actual: 0 1 :prediction preds_proba: [[0.49756566 0.50243434]] 	 result: 

2016-11-17 	 actual: 0 0 :prediction preds_proba: [[0.50196247 0.49803753]] 	 result: 16.0
2016-11-16 	 actual: 0 0 :prediction preds_proba: [[0.50115447 0.49884553]] 	 result: 3.0
2016-11-15 	 actual: 1 1 :prediction preds_proba: [[0.4981818 0.5018182]] 	 result: 3.0
2016-11-14 	 actual: 0 0 :prediction preds_proba: [[0.50500845 0.49499155]] 	 result: 51.0
2016-11-10 	 actual: 0 0 :prediction preds_proba: [[0.50415442 0.49584558]] 	 result: 14.0
2016-11-09 	 actual: 1 0 :prediction preds_proba: [[0.50118861 0.49881139]] 	 result: -43.0
2016-11-08 	 actual: 1 1 :prediction preds_proba: [[0.4998092 0.5001908]] 	 result: 17.0
2016-11-07 	 actual: 0 0 :prediction preds_proba: [[0.50265144 0.49734856]] 	 result: 12.0
2016-11-04 	 actual: 0 0 :prediction preds_proba: [[0.50174532 0.49825468]] 	 result: 1.0
2016-11-03 	 actual: 1 1 :prediction preds_proba: [[0.49977493 0.50022507]] 	 result: 3.0
2016-11-02 	 actual: 0 0 :prediction preds_proba: [[0.50551677 0.49448323]] 	 result: 39.0
2016-1

...

2013-08-07 	 actual: 1 1 :prediction preds_proba: [[0.49804639 0.50195361]] 	 result: 20.0
2013-08-06 	 actual: 0 1 :prediction preds_proba: [[0.4988706 0.5011294]] 	 result: -42.0
2013-08-05 	 actual: 0 1 :prediction preds_proba: [[0.49884787 0.50115213]] 	 result: -3.0
2013-08-02 	 actual: 1 1 :prediction preds_proba: [[0.49974469 0.50025531]] 	 result: 51.0
2013-08-01 	 actual: 1 1 :prediction preds_proba: [[0.49989696 0.50010304]] 	 result: 8.0
2013-07-31 	 actual: 1 0 :prediction preds_proba: [[0.50248184 0.49751816]] 	 result: -43.0
2013-07-30 	 actual: 1 0 :prediction preds_proba: [[0.50167271 0.49832729]] 	 result: -4.0
2013-07-29 	 actual: 1 1 :prediction preds_proba: [[0.49889241 0.50110759]] 	 result: 8.0
2013-07-26 	 actual: 0 0 :prediction preds_proba: [[0.50074389 0.49925611]] 	 result: 17.0
2013-07-25 	 actual: 0 0 :prediction preds_proba: [[0.50261627 0.49738373]] 	 result: 7.0
2013-07-24 	 actual: 1 0 :prediction preds_proba: [[0.50261195 0.49738805]] 	 result: -7.0
20

2013-01-21 	 actual: 1 0 :prediction preds_proba: [[0.50083816 0.49916184]] 	 result: -6.0
2013-01-18 	 actual: 0 0 :prediction preds_proba: [[0.50092112 0.49907888]] 	 result: 13.0
2013-01-17 	 actual: 1 0 :prediction preds_proba: [[0.50370655 0.49629345]] 	 result: -12.0
2013-01-16 	 actual: 1 1 :prediction preds_proba: [[0.49956653 0.50043347]] 	 result: 38.0
2013-01-15 	 actual: 0 0 :prediction preds_proba: [[0.50003448 0.49996552]] 	 result: 18.0
2013-01-14 	 actual: 0 1 :prediction preds_proba: [[0.49852265 0.50147735]] 	 result: -5.0
2013-01-11 	 actual: 0 0 :prediction preds_proba: [[0.50164226 0.49835774]] 	 result: 17.0
2013-01-10 	 actual: 0 1 :prediction preds_proba: [[0.49893149 0.50106851]] 	 result: -9.0
2013-01-09 	 actual: 0 0 :prediction preds_proba: [[0.50001073 0.49998927]] 	 result: 2.0
2013-01-08 	 actual: 0 0 :prediction preds_proba: [[0.50006802 0.49993198]] 	 result: 8.0
2013-01-07 	 actual: 0 1 :prediction preds_proba: [[0.49923076 0.50076924]] 	 result: -22.0

2012-07-02 	 actual: 0 0 :prediction preds_proba: [[0.50367796 0.49632204]] 	 result: 12.0
2012-06-29 	 actual: 0 0 :prediction preds_proba: [[0.50277232 0.49722768]] 	 result: 0.0
2012-06-28 	 actual: 0 0 :prediction preds_proba: [[0.50087012 0.49912988]] 	 result: 40.0
2012-06-27 	 actual: 1 1 :prediction preds_proba: [[0.49912049 0.50087951]] 	 result: 12.0
2012-06-26 	 actual: 1 0 :prediction preds_proba: [[0.50194983 0.49805017]] 	 result: -33.0
2012-06-25 	 actual: 0 0 :prediction preds_proba: [[0.50125209 0.49874791]] 	 result: 12.0
2012-06-22 	 actual: 1 0 :prediction preds_proba: [[0.50268503 0.49731497]] 	 result: -12.0
2012-06-21 	 actual: 0 1 :prediction preds_proba: [[0.49986042 0.50013958]] 	 result: -3.0
2012-06-20 	 actual: 0 0 :prediction preds_proba: [[0.50322351 0.49677649]] 	 result: 15.0
2012-06-19 	 actual: 1 1 :prediction preds_proba: [[0.49472325 0.50527675]] 	 result: 56.0
2012-06-18 	 actual: 0 0 :prediction preds_proba: [[0.5013798 0.4986202]] 	 result: 24.0


2011-12-14 	 actual: 0 0 :prediction preds_proba: [[0.50551723 0.49448277]] 	 result: 7.0
2011-12-13 	 actual: 1 0 :prediction preds_proba: [[0.50492031 0.49507969]] 	 result: -27.0
2011-12-12 	 actual: 0 0 :prediction preds_proba: [[0.50692419 0.49307581]] 	 result: 54.0
2011-12-09 	 actual: 1 0 :prediction preds_proba: [[0.50237555 0.49762445]] 	 result: -46.0
2011-12-08 	 actual: 0 0 :prediction preds_proba: [[0.50210736 0.49789264]] 	 result: 57.0
2011-12-07 	 actual: 0 1 :prediction preds_proba: [[0.49882152 0.50117848]] 	 result: -5.0
2011-12-06 	 actual: 0 0 :prediction preds_proba: [[0.50361906 0.49638094]] 	 result: 27.0
2011-12-05 	 actual: 1 1 :prediction preds_proba: [[0.4989309 0.5010691]] 	 result: 10.0
2011-12-02 	 actual: 0 1 :prediction preds_proba: [[0.48728953 0.51271047]] 	 result: -15.0
2011-12-01 	 actual: 0 1 :prediction preds_proba: [[0.4986212 0.5013788]] 	 result: -22.0
2011-11-30 	 actual: 1 0 :prediction preds_proba: [[0.50323666 0.49676334]] 	 result: -115.

2011-05-27 	 actual: 1 0 :prediction preds_proba: [[0.50162354 0.49837646]] 	 result: -12.0
2011-05-26 	 actual: 0 0 :prediction preds_proba: [[0.50474612 0.49525388]] 	 result: 12.0
2011-05-25 	 actual: 1 1 :prediction preds_proba: [[0.49899332 0.50100668]] 	 result: 47.0
2011-05-24 	 actual: 1 0 :prediction preds_proba: [[0.50098615 0.49901385]] 	 result: -8.0
2011-05-23 	 actual: 0 1 :prediction preds_proba: [[0.49853818 0.50146182]] 	 result: -22.0
2011-05-20 	 actual: 0 0 :prediction preds_proba: [[0.50184692 0.49815308]] 	 result: 4.0
2011-05-19 	 actual: 1 0 :prediction preds_proba: [[0.50123911 0.49876089]] 	 result: -13.0
2011-05-18 	 actual: 1 1 :prediction preds_proba: [[0.4993407 0.5006593]] 	 result: 7.0
2011-05-17 	 actual: 0 0 :prediction preds_proba: [[0.50107516 0.49892484]] 	 result: 22.0
2011-05-16 	 actual: 1 1 :prediction preds_proba: [[0.49687624 0.50312376]] 	 result: 30.0
2011-05-13 	 actual: 0 0 :prediction preds_proba: [[0.50126901 0.49873099]] 	 result: 20.0


### 6. Voting Classifier

Finally, there was Voting Classifier used for 3 estimators with best results - Logistic Regression, KNeighbors and Random Forest. Majority rule for voting was applied.

In [28]:
train_period = 1000
test_period = 2000
date, predictions, y_actual, results = [], [], [], []

lr_clf = LogisticRegression(C=0.001)
knn_clf = KNeighborsClassifier(n_neighbors=200)
rfc_clf = RandomForestClassifier(n_estimators=100, max_depth=3)
voting_clf = VotingClassifier(
    estimators = [('lr', lr_clf),('knn_clf', knn_clf),('rfc_clf', rfc_clf)],
    voting = 'hard')

for t in range(test_period):
    X_train = data_fw20_all.iloc[(-train_period - t - 1) : (-t - 1)][['Close_ret_-1', 'Volume_ret_-1', 
                                                                      'OpenInt_ret_-1']]
    y_train = data_fw20_all.iloc[(-train_period - t - 1) : (-t - 1)][['FW20_dir_o_c']]
    X_test = data_fw20_all.iloc[-t - 1][['Close_ret_-1', 'Volume_ret_-1', 'OpenInt_ret_-1']]
    y_test = data_fw20_all.iloc[-t - 1]['FW20_dir_o_c']

    voting_clf.fit(X_train, y_train)
    prediction = voting_clf.predict(np.array(X_test).reshape(1,-1))
    date.append(np.datetime64(data_fw20_all.index[-t -1], 'D'))
    y_actual.append(int(y_test))
    predictions.append(prediction[0])
    
    result = abs(data_fw20_all.iloc[-t - 1]['Close'] - data_fw20_all.iloc[-t - 1]['Open'])
    results.append(result if y_test == prediction else -result)

accuracy = 0
for i in range(len(predictions)):
    if y_actual[i] == predictions[i]:
        accuracy+=1
print('Model summary')
print('Accuracy: {}'.format(round(accuracy / len(predictions),4)))
print('Result (pts.): {}'.format(sum(results)))
print('Average result per 1 session: {}'.format(sum(results) / test_period))
print('*'*100)

for i in range(len(predictions)):
    print(date[i], '\t', 'actual:', y_actual[i], predictions[i], ':prediction', '\t', 'result:', results[i])

Model summary
Accuracy: 0.5355
Result (pts.): 2650.0
Average result per 1 session: 1.325
****************************************************************************************************
2018-12-28 	 actual: 1 0 :prediction 	 result: -16.0
2018-12-27 	 actual: 0 0 :prediction 	 result: 40.0
2018-12-21 	 actual: 0 0 :prediction 	 result: 38.0
2018-12-20 	 actual: 0 0 :prediction 	 result: 10.0
2018-12-19 	 actual: 1 0 :prediction 	 result: -33.0
2018-12-18 	 actual: 1 0 :prediction 	 result: -46.0
2018-12-17 	 actual: 0 0 :prediction 	 result: 22.0
2018-12-14 	 actual: 1 0 :prediction 	 result: -13.0
2018-12-13 	 actual: 1 0 :prediction 	 result: -21.0
2018-12-12 	 actual: 1 0 :prediction 	 result: -53.0
2018-12-11 	 actual: 0 0 :prediction 	 result: 48.0
2018-12-10 	 actual: 0 0 :prediction 	 result: 30.0
2018-12-07 	 actual: 0 0 :prediction 	 result: 25.0
2018-12-06 	 actual: 0 0 :prediction 	 result: 29.0
2018-12-05 	 actual: 1 0 :prediction 	 result: -38.0
2018-12-04 	 actual: 1 

2018-03-28 	 actual: 0 0 :prediction 	 result: 21.0
2018-03-27 	 actual: 0 0 :prediction 	 result: 19.0
2018-03-26 	 actual: 0 0 :prediction 	 result: 20.0
2018-03-23 	 actual: 1 0 :prediction 	 result: -6.0
2018-03-22 	 actual: 0 0 :prediction 	 result: 36.0
2018-03-21 	 actual: 1 0 :prediction 	 result: -46.0
2018-03-20 	 actual: 1 0 :prediction 	 result: -12.0
2018-03-19 	 actual: 0 0 :prediction 	 result: 45.0
2018-03-16 	 actual: 0 0 :prediction 	 result: 40.0
2018-03-15 	 actual: 0 0 :prediction 	 result: 7.0
2018-03-14 	 actual: 0 0 :prediction 	 result: 13.0
2018-03-13 	 actual: 0 0 :prediction 	 result: 27.0
2018-03-12 	 actual: 1 0 :prediction 	 result: -8.0
2018-03-09 	 actual: 1 0 :prediction 	 result: -36.0
2018-03-08 	 actual: 0 0 :prediction 	 result: 42.0
2018-03-07 	 actual: 1 0 :prediction 	 result: -36.0
2018-03-06 	 actual: 0 0 :prediction 	 result: 8.0
2018-03-05 	 actual: 1 0 :prediction 	 result: -32.0
2018-03-02 	 actual: 0 0 :prediction 	 result: 12.0
2018-03-0

2017-07-28 	 actual: 1 0 :prediction 	 result: -22.0
2017-07-27 	 actual: 0 0 :prediction 	 result: 10.0
2017-07-26 	 actual: 1 0 :prediction 	 result: -8.0
2017-07-25 	 actual: 1 0 :prediction 	 result: -1.0
2017-07-24 	 actual: 0 0 :prediction 	 result: 10.0
2017-07-21 	 actual: 0 1 :prediction 	 result: -14.0
2017-07-20 	 actual: 0 0 :prediction 	 result: 34.0
2017-07-19 	 actual: 1 0 :prediction 	 result: -8.0
2017-07-18 	 actual: 0 0 :prediction 	 result: 3.0
2017-07-17 	 actual: 1 0 :prediction 	 result: -22.0
2017-07-14 	 actual: 0 0 :prediction 	 result: 3.0
2017-07-13 	 actual: 1 1 :prediction 	 result: 15.0
2017-07-12 	 actual: 1 0 :prediction 	 result: -43.0
2017-07-11 	 actual: 0 1 :prediction 	 result: -10.0
2017-07-10 	 actual: 0 0 :prediction 	 result: 13.0
2017-07-07 	 actual: 0 1 :prediction 	 result: -1.0
2017-07-06 	 actual: 0 0 :prediction 	 result: 7.0
2017-07-05 	 actual: 1 0 :prediction 	 result: -5.0
2017-07-04 	 actual: 0 0 :prediction 	 result: 14.0
2017-07-03

2016-11-29 	 actual: 1 0 :prediction 	 result: -2.0
2016-11-28 	 actual: 0 0 :prediction 	 result: 16.0
2016-11-25 	 actual: 0 0 :prediction 	 result: 15.0
2016-11-24 	 actual: 1 1 :prediction 	 result: 6.0
2016-11-23 	 actual: 1 0 :prediction 	 result: -17.0
2016-11-22 	 actual: 1 0 :prediction 	 result: -4.0
2016-11-21 	 actual: 1 0 :prediction 	 result: -18.0
2016-11-18 	 actual: 0 0 :prediction 	 result: 11.0
2016-11-17 	 actual: 0 0 :prediction 	 result: 16.0
2016-11-16 	 actual: 0 0 :prediction 	 result: 3.0
2016-11-15 	 actual: 1 0 :prediction 	 result: -3.0
2016-11-14 	 actual: 0 0 :prediction 	 result: 51.0
2016-11-10 	 actual: 0 1 :prediction 	 result: -14.0
2016-11-09 	 actual: 1 0 :prediction 	 result: -43.0
2016-11-08 	 actual: 1 0 :prediction 	 result: -17.0
2016-11-07 	 actual: 0 0 :prediction 	 result: 12.0
2016-11-04 	 actual: 0 0 :prediction 	 result: 1.0
2016-11-03 	 actual: 1 1 :prediction 	 result: 3.0
2016-11-02 	 actual: 0 0 :prediction 	 result: 39.0
2016-10-31 

2016-04-01 	 actual: 0 0 :prediction 	 result: 7.0
2016-03-31 	 actual: 0 0 :prediction 	 result: 1.0
2016-03-30 	 actual: 1 0 :prediction 	 result: -39.0
2016-03-29 	 actual: 1 0 :prediction 	 result: -4.0
2016-03-24 	 actual: 1 0 :prediction 	 result: -8.0
2016-03-23 	 actual: 0 0 :prediction 	 result: 38.0
2016-03-22 	 actual: 1 1 :prediction 	 result: 15.0
2016-03-21 	 actual: 1 0 :prediction 	 result: -1.0
2016-03-18 	 actual: 1 0 :prediction 	 result: -13.0
2016-03-17 	 actual: 1 0 :prediction 	 result: -22.0
2016-03-16 	 actual: 0 1 :prediction 	 result: -5.0
2016-03-15 	 actual: 0 0 :prediction 	 result: 2.0
2016-03-14 	 actual: 0 0 :prediction 	 result: 5.0
2016-03-11 	 actual: 1 1 :prediction 	 result: 20.0
2016-03-10 	 actual: 1 0 :prediction 	 result: -11.0
2016-03-09 	 actual: 1 1 :prediction 	 result: 7.0
2016-03-08 	 actual: 0 0 :prediction 	 result: 12.0
2016-03-07 	 actual: 1 0 :prediction 	 result: -15.0
2016-03-04 	 actual: 0 0 :prediction 	 result: 4.0
2016-03-03 	 

...

2014-11-27 	 actual: 1 0 :prediction 	 result: -1.0
2014-11-26 	 actual: 1 0 :prediction 	 result: -10.0
2014-11-25 	 actual: 1 1 :prediction 	 result: 11.0
2014-11-24 	 actual: 0 0 :prediction 	 result: 25.0
2014-11-21 	 actual: 1 0 :prediction 	 result: -17.0
2014-11-20 	 actual: 0 0 :prediction 	 result: 12.0
2014-11-19 	 actual: 0 0 :prediction 	 result: 13.0
2014-11-18 	 actual: 1 1 :prediction 	 result: 4.0
2014-11-17 	 actual: 1 0 :prediction 	 result: -8.0
2014-11-14 	 actual: 1 0 :prediction 	 result: -27.0
2014-11-13 	 actual: 0 0 :prediction 	 result: 18.0
2014-11-12 	 actual: 1 0 :prediction 	 result: -6.0
2014-11-10 	 actual: 0 0 :prediction 	 result: 9.0
2014-11-07 	 actual: 0 0 :prediction 	 result: 21.0
2014-11-06 	 actual: 0 0 :prediction 	 result: 13.0
2014-11-05 	 actual: 0 0 :prediction 	 result: 24.0
2014-11-04 	 actual: 1 1 :prediction 	 result: 5.0
2014-11-03 	 actual: 0 0 :prediction 	 result: 1.0
2014-10-31 	 actual: 0 0 :prediction 	 result: 10.0
2014-10-30 	 

2014-02-20 	 actual: 1 0 :prediction 	 result: -3.0
2014-02-19 	 actual: 1 0 :prediction 	 result: -19.0
2014-02-18 	 actual: 0 0 :prediction 	 result: 15.0
2014-02-17 	 actual: 0 0 :prediction 	 result: 4.0
2014-02-14 	 actual: 0 0 :prediction 	 result: 8.0
2014-02-13 	 actual: 1 0 :prediction 	 result: -17.0
2014-02-12 	 actual: 1 0 :prediction 	 result: -31.0
2014-02-11 	 actual: 1 0 :prediction 	 result: -26.0
2014-02-10 	 actual: 0 1 :prediction 	 result: -18.0
2014-02-07 	 actual: 0 0 :prediction 	 result: 1.0
2014-02-06 	 actual: 1 0 :prediction 	 result: -42.0
2014-02-05 	 actual: 0 0 :prediction 	 result: 12.0
2014-02-04 	 actual: 1 0 :prediction 	 result: -18.0
2014-02-03 	 actual: 1 0 :prediction 	 result: -26.0
2014-01-31 	 actual: 1 0 :prediction 	 result: -7.0
2014-01-30 	 actual: 1 0 :prediction 	 result: -28.0
2014-01-29 	 actual: 0 0 :prediction 	 result: 40.0
2014-01-28 	 actual: 1 0 :prediction 	 result: -9.0
2014-01-27 	 actual: 0 0 :prediction 	 result: 15.0
2014-0

2012-11-21 	 actual: 0 0 :prediction 	 result: 19.0
2012-11-20 	 actual: 1 0 :prediction 	 result: -4.0
2012-11-19 	 actual: 1 1 :prediction 	 result: 20.0
2012-11-16 	 actual: 0 0 :prediction 	 result: 5.0
2012-11-15 	 actual: 1 0 :prediction 	 result: -11.0
2012-11-14 	 actual: 1 0 :prediction 	 result: -21.0
2012-11-13 	 actual: 1 0 :prediction 	 result: -22.0
2012-11-12 	 actual: 1 0 :prediction 	 result: -20.0
2012-11-09 	 actual: 0 0 :prediction 	 result: 9.0
2012-11-08 	 actual: 1 0 :prediction 	 result: -3.0
2012-11-07 	 actual: 0 0 :prediction 	 result: 20.0
2012-11-06 	 actual: 0 0 :prediction 	 result: 10.0
2012-11-05 	 actual: 1 1 :prediction 	 result: 13.0
2012-11-02 	 actual: 1 0 :prediction 	 result: -9.0
2012-10-31 	 actual: 0 1 :prediction 	 result: -18.0
2012-10-30 	 actual: 1 1 :prediction 	 result: 3.0
2012-10-29 	 actual: 0 0 :prediction 	 result: 6.0
2012-10-26 	 actual: 1 0 :prediction 	 result: -22.0
2012-10-25 	 actual: 0 1 :prediction 	 result: -13.0
2012-10-2

2012-03-21 	 actual: 0 0 :prediction 	 result: 7.0
2012-03-20 	 actual: 0 1 :prediction 	 result: -22.0
2012-03-19 	 actual: 0 0 :prediction 	 result: 8.0
2012-03-16 	 actual: 0 0 :prediction 	 result: 11.0
2012-03-15 	 actual: 1 0 :prediction 	 result: -6.0
2012-03-14 	 actual: 1 0 :prediction 	 result: -15.0
2012-03-13 	 actual: 1 0 :prediction 	 result: -8.0
2012-03-12 	 actual: 1 1 :prediction 	 result: 5.0
2012-03-09 	 actual: 0 0 :prediction 	 result: 16.0
2012-03-08 	 actual: 1 0 :prediction 	 result: -33.0
2012-03-07 	 actual: 1 0 :prediction 	 result: -11.0
2012-03-06 	 actual: 0 0 :prediction 	 result: 42.0
2012-03-05 	 actual: 0 0 :prediction 	 result: 31.0
2012-03-02 	 actual: 1 1 :prediction 	 result: 4.0
2012-03-01 	 actual: 1 0 :prediction 	 result: -11.0
2012-02-29 	 actual: 0 0 :prediction 	 result: 23.0
2012-02-28 	 actual: 0 1 :prediction 	 result: -1.0
2012-02-27 	 actual: 1 0 :prediction 	 result: -5.0
2012-02-24 	 actual: 0 0 :prediction 	 result: 6.0
2012-02-23 	

2011-06-09 	 actual: 1 0 :prediction 	 result: -3.0
2011-06-08 	 actual: 0 0 :prediction 	 result: 8.0
2011-06-07 	 actual: 1 1 :prediction 	 result: 13.0
2011-06-06 	 actual: 0 0 :prediction 	 result: 8.0
2011-06-03 	 actual: 0 0 :prediction 	 result: 1.0
2011-06-02 	 actual: 0 0 :prediction 	 result: 2.0
2011-06-01 	 actual: 0 0 :prediction 	 result: 11.0
2011-05-31 	 actual: 1 1 :prediction 	 result: 30.0
2011-05-30 	 actual: 1 0 :prediction 	 result: -1.0
2011-05-27 	 actual: 1 0 :prediction 	 result: -12.0
2011-05-26 	 actual: 0 0 :prediction 	 result: 12.0
2011-05-25 	 actual: 1 1 :prediction 	 result: 47.0
2011-05-24 	 actual: 1 0 :prediction 	 result: -8.0
2011-05-23 	 actual: 0 0 :prediction 	 result: 22.0
2011-05-20 	 actual: 0 0 :prediction 	 result: 4.0
2011-05-19 	 actual: 1 0 :prediction 	 result: -13.0
2011-05-18 	 actual: 1 0 :prediction 	 result: -7.0
2011-05-17 	 actual: 0 0 :prediction 	 result: 22.0
2011-05-16 	 actual: 1 1 :prediction 	 result: 30.0
2011-05-13 	 ac

## Conclusions

Using simple classifiers may give some slight advantage over the market. Results obtained in the analysis show that all of the tested algorithms have accuracy over 50%. The highest accuracy was obtained for Random Forest (about 54%). Unfortunately, Voting Classifier didn't boost the results compared to best single estimator (Random Forest).

Suggestions for further analysis:
- wider interval (week or even month) - daytrading implies high transaction costs,
- application of other predictors - for example rates of return from other correlated markets (SPX, NDX, DAX, NKX),
- reducing the number of signals by selecting predictions with highest probability.